<h1>Sistema de Reconocimiento Facial Emocional</h1>

## 1. Importamos las librerias y paquetes requeridos

In [ ]:
# librerias a utilizar
import cv2
import os
import time
import mediapipe as mp
import imutils
import numpy as np
from keras.models import load_model
import json
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.models import Sequential #Se utiliza para crear modelos de redes neuronales secuenciales (Es aquel en el que las capas se apilan una encima de la otra)
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam #Se utiliza para ajustar los pesos y los sesgos de las redes neuronales durante el entrenamiento
from keras.preprocessing.image import ImageDataGenerator #Permite generar imagenes aumentadas en tiempo real durante el entrenamiento, ayuda a aumentar el tamaño y la diversidad del conjunto de datos lo que mejora la capacidad de generalización del modelo
"""
Capas de las redes neuronales:
Conv2D - Se realiza la convulsión en imagenes de entrada para extraer características/patrones importantes
MaxPooling2D - Reduce la dimensionalidad y extrae características dominantes
Dense - Es una capa completamente conectada, cada entrada se conecta con todos los nodos de salida
Dropout - Esta capa se utiliza para regularizar el modelo y evitar el sobreajuste, apaga neuronas aleatoriamente durante el entrenamiento para evitar la dependencia excesiva
Flatten - Aplana los datos multidimensionales en una matriz unidimensional antes de pasarlo a las capas completamente conectadas
"""

## 2. Método para Almacenar Rostro del usuario

La función almacenarRostro(personName) tiene como objetivo capturar un video utilizando la cámara del dispositivo y almacenarlo en formato mp4. A continuación, se proporciona una descripción de lo que hace la función y los elementos que requiere:

personName: Parámetro que indica el nombre del archivo de video a almacenar. Este nombre se utilizará para generar la ruta completa del archivo de video.
La función comienza pausando la ejecución durante 3 segundos utilizando time.sleep(3). Esto puede ser útil para permitir que el dispositivo se prepare adecuadamente antes de comenzar a grabar el video.

A continuación, se establece la ruta de almacenamiento del video utilizando dataPath_mp4 y el nombre de la persona proporcionado. El video se almacenará en la carpeta especificada.

Luego, se verifica si ya existe un video con el mismo nombre en la ruta especificada. Si no existe, se procede a capturar el video utilizando la cámara del dispositivo.

Dentro del bucle while, se obtienen los frames de video de la cámara utilizando captura.read(). Luego, se aplica un efecto espejo a la imagen utilizando cv2.flip(imagen, 1). Si se detecta correctamente un frame de video, se muestra en una ventana titulada "video" utilizando cv2.imshow('video', imagen). Además, el frame de video se escribe en el archivo de salida utilizando salida.write(imagen).

Si se presiona la tecla 's', el bucle se interrumpe y se finaliza la captura de video. El archivo de video se guarda en la ubicación especificada.

Si ya existe un video con el mismo nombre en la ruta especificada, se muestra un mensaje indicando que ya existe un video con ese nombre.

In [2]:
def almacenarRostro(personName):
    time.sleep(3)
    dataPath_mp4 = 'C:/Users/iamra/FacialRecognition/Videos' #Cambia a la ruta donde hayas almacenado Data
    videoPath = dataPath_mp4 + '/' + personName + '.mp4' #Ruta completa

    if not os.path.exists(videoPath): # Si no existe el video con ese nombre lo crea
        captura = cv2.VideoCapture(0) # Definioms que usaremos la camara
        salida = cv2.VideoWriter(videoPath,cv2.VideoWriter_fourcc(*'XVID'),20.0,(640,480)) # Cambiar con la otra camara
        while True:
            ret, imagen = captura.read() # Comienza a grabar
            imagen = cv2.flip(imagen, 1) # Aplicamos efecto espejo
            if ret == True: # Si detecta video
                cv2.imshow('video', imagen) # Mostramos la imagen
                salida.write(imagen) # Comenzamos a guardar la imagen
                if cv2.waitKey(1) & 0xFF == ord('s'): # Si se presiona la tecla s, paramos de grabar y se guarda el video
                      break
            else: break
        captura.release()
        salida.release()
        cv2.destroyAllWindows()

    else: 
        print("Ya existe un video con ese nombre")

## 3. Método para almacenar Datos del usuario

La función almacenarDatos(personName, cargo, edad) tiene como objetivo almacenar información personal en un archivo de texto. A continuación, se proporciona una descripción de lo que hace la función y los elementos que requiere:

personName: Parámetro que indica el nombre de la persona cuyos datos se van a almacenar.
cargo: Parámetro que indica el cargo de la persona.
edad: Parámetro que indica la edad de la persona.
La función comienza estableciendo la ruta de almacenamiento del archivo de texto utilizando dataPath_txt y el nombre de la persona proporcionado. El archivo de texto se almacenará en la carpeta especificada.

A continuación, se verifica si ya existe un archivo con el mismo nombre en la ruta especificada. Si no existe, se procede a crear el archivo de texto y escribir los datos en él.

Dentro del bloque with open(filePath, "w") as file:, se abre el archivo en modo de escritura ("w"). Luego, se escriben los datos de la persona en líneas separadas utilizando file.write(). Los datos que se escriben son el nombre, el cargo y la edad de la persona.

Si ya existe un archivo con el mismo nombre en la ruta especificada, se muestra un mensaje indicando que ya existe un archivo con esos datos.

In [25]:
def almacenarDatos(personName, cargo, edad):
    dataPath_txt = 'C:/Users/iamra/FacialRecognition/Datos' # Cambiar en la otra computadora
    filePath = dataPath_txt + '/' + personName + '.txt'

    if not os.path.exists(filePath):
        with open(filePath, "w") as file:
            file.write("Nombre: " + personName + os.linesep)
            file.write("Cargo: " + cargo +  os.linesep) 
            file.write("Edad: " + edad)

    else:
        print("Ya existe un archivo con estos datos")


## 4. Método para generar un informe con los usuarios registrados 

La función generarInformes() tiene como objetivo recopilar y mostrar los datos almacenados en archivos de texto previamente guardados. A continuación, se proporciona una descripción de lo que hace la función:

La función comienza estableciendo la ruta de acceso a la carpeta que contiene los archivos de texto utilizando dataPath_txt.

A continuación, se inicializan las listas Datos, Nombres, Cargos y Edades, que se utilizarán para almacenar la información extraída de los archivos de texto.

Se obtiene la lista de archivos presentes en la carpeta utilizando os.listdir(dataPath_txt). Luego, se itera sobre cada archivo de la lista en el bucle for person in peopleList.

Dentro del bucle, se construye la ruta completa del archivo de texto utilizando filePathPerson. Luego, se abre el archivo en modo de lectura ("r") utilizando with open(filePathPerson, "r") as file:. A continuación, se lee el contenido del archivo utilizando file.readlines() y se agrega a la lista Datos.

Posteriormente, se itera sobre cada elemento de la lista Datos en el bucle for Dato in Datos. Dentro del bucle, se divide cada línea en palabras utilizando split() y se extrae el nombre, cargo y edad correspondientes. Estos valores se agregan a las listas Nombres, Cargos y Edades respectivamente.

Finalmente, se imprimen en la consola las listas Nombres, Cargos y Edades, lo que muestra los datos recopilados de los archivos de texto.

In [15]:
def generarInformes():
    dataPath_txt = 'C:/Users/iamra/FacialRecognition/Datos'
    Datos = []
    Nombres = []
    Cargos = []
    Edades = []

    peopleList = os.listdir(dataPath_txt)
    for person in peopleList: 
        filePathPerson = dataPath_txt + '/' + person
        with open(filePathPerson, "r") as file:
            Datos.append(file.readlines())


    for Dato in Datos:
        aux = Dato[0].split()
        Nombres.append(aux[1])
        aux = Dato[2].split()
        Cargos.append(aux[1])
        aux = Dato[4].split()
        Edades.append(aux[1])

    print(Nombres)
    print(Cargos)
    print(Edades)

## 5. Método para extraer los rostros 

La función extraerRostro(personName) tiene como objetivo extraer y almacenar rostros de un video específico en una carpeta designada. A continuación, se proporciona una descripción de lo que hace la función:

La función comienza estableciendo la ruta de almacenamiento de los rostros en dataPath, y crea una subcarpeta para la persona específica utilizando personPath.

Se verifica si la carpeta de la persona ya existe en la ruta especificada. Si no existe, se crea la carpeta utilizando os.makedirs(personPath). Además, se imprime en la consola un mensaje indicando la creación de la carpeta.

A continuación, se utiliza el detector de rostros de MediaPipe (mp_face_detection) y la utilidad de dibujo (mp_drawing) para detectar y dibujar recuadros alrededor de los rostros en el video.

Se abre el video utilizando cv2.VideoCapture, especificando la ruta del video correspondiente al nombre de la persona proporcionado.

Se inicializa una variable de contador (count) que llevará la cuenta de las fotos (rostros) que se han tomado.

Dentro del bucle while, se lee cada frame del video utilizando cap.read(). Si no se puede leer correctamente, se sale del bucle.

Se redimensiona el frame a un ancho de 640 píxeles utilizando imutils.resize(frame, width=640). Luego, se obtienen las dimensiones del frame utilizando frame.shape.

Se copia el frame en una variable auxiliar (aux_frame) para trabajar con ella. Se convierte el frame de BGR a RGB utilizando cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).

Se procesa el frame con el detector de rostros utilizando face_detection.process(frame_rgb). Si se detecta uno o más rostros en el frame, se entra en el bucle for y se realizan las siguientes acciones:

Se dibuja un recuadro alrededor de cada rostro y se resaltan los puntos de interés utilizando mp_drawing.draw_detection(frame, detection, ...).
Se obtienen las coordenadas del rostro en el frame utilizando detection.location_data.relative_bounding_box.
Se recorta y ajusta el tamaño del rostro utilizando las coordenadas obtenidas.
Se guarda el rostro en la carpeta correspondiente utilizando cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count), rostro).
Se muestra el rostro guardado en una ventana separada utilizando cv2.imshow('rostro', rostro).
Se incrementa el contador de fotos (count).
Además, se muestra el frame en una ventana utilizando cv2.imshow("frame", frame).

El bucle continúa hasta que se presione la tecla 'Esc' o hasta que se alcance un límite de 300 rostros guardados.

Finalmente, se libera el video y se cierran las ventanas utilizando cap.release() y cv2.destroyAllWindows() respectivamente.

In [14]:
def extraerRostro(personName):
    dataPath = 'C:/Users/iamra/FacialRecognition/Rostros'# Lugar donde se van a almacenar los rostros
    personPath = dataPath + '/' + personName 

    if not os.path.exists(personPath): # Verifica si existe la carpete de la persona si no crea una
        print('Carpeta creada: ',personPath)
        os.makedirs(personPath)

    mp_face_detection = mp.solutions.face_detection # Es el detector de rostros
    mp_drawing = mp.solutions.drawing_utils # Esto es para colocar el recuadro y los puntos
    cap = cv2.VideoCapture('C:/Users/iamra/FacialRecognition/Videos/' + personName + '.mp4')

    count = 0 # Esta variable va a llevar la cuenta de las fotos que se han tomado

    with mp_face_detection.FaceDetection(  # Gestor de contexto
        min_detection_confidence=0.6) as face_detection:
        while True:
            ret, frame = cap.read() # Comenzamos a leer el video
            if ret == False: # Verificamos si se esta leyendo de forma correcta
                break
            frame = imutils.resize(frame, width=640)
            height, width, _ = frame.shape # Obtenemos las dimensipones de la imagen
            aux_frame = frame.copy() # Copiamos el frame 
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2 lo lee como BGR y lo convertimos a RGB       
            results = face_detection.process(frame_rgb) # Obtenemos las cordenadas de la cara y de los puntos de interes
            k = cv2.waitKey(1) & 0xFF
            if results.detections is not None: # Si detecta un rostro entra aqui 
                for detection in results.detections: # Se usa un for en caso de detectar mas de un rostro
                    mp_drawing.draw_detection(frame, detection,
                        mp_drawing.DrawingSpec(color=(0, 255, 255), circle_radius=2),
                        mp_drawing.DrawingSpec(color=(255, 0, 255))) # Dibuja en un recuadro un rostro y con circulos los puntos de interes

                    # Obtenemos la posicion de la cara
                    x = int(detection.location_data.relative_bounding_box.xmin * width)
                    y = int(detection.location_data.relative_bounding_box.ymin * height)
                    w = int(detection.location_data.relative_bounding_box.width * width)
                    h = int(detection.location_data.relative_bounding_box.height * height)

                    # Recortamos el rostro y reajustamos el tamaño
                    rostro = aux_frame[y:y+h,x:x+w]
                    rostro = cv2.resize(rostro,(150,150), interpolation=cv2.INTER_CUBIC)
                    cv2.imwrite(personPath + '/rotro_{}.jpg'.format(count),rostro)
                    cv2.imshow('rostro',rostro) # Mostramos el rostro guardado 
                    count = count +1 

            cv2.imshow("frame", frame) # Muestra lo que esta detectando 
            if k == 27 or count >= 300: # Si tecleamos la tecla esc se cancela o si llega a 300
                break
    cap.release()
    cv2.destroyAllWindows()

## 6. Método para entrenar el Modelo de Reconocimiento Facial

La función entrenarMRF(modelName) tiene como objetivo entrenar un modelo de reconocimiento facial utilizando el algoritmo de reconocimiento LBPH (Local Binary Patterns Histograms). A continuación, se proporciona una descripción de lo que hace la función:

La función comienza estableciendo la ruta de acceso a la carpeta que contiene los rostros de entrenamiento en dataPath. Luego, se obtiene una lista de los nombres de las personas presentes en la carpeta utilizando os.listdir(dataPath).

Se inicializan las listas labels y facesData, que se utilizarán para almacenar las etiquetas y los datos de los rostros respectivamente.

Se inicializa la variable label en 0, que se utilizará para asignar etiquetas a los rostros.

Se itera sobre cada nombre de directorio en la lista peopleList utilizando el bucle for nameDir in peopleList. Dentro del bucle, se construye la ruta completa de la carpeta de la persona utilizando personPath.

Se itera sobre cada nombre de archivo en la carpeta de la persona utilizando os.listdir(personPath). Dentro del bucle, se agrega la etiqueta correspondiente a la lista labels, y se lee la imagen del rostro en escala de grises utilizando cv2.imread(personPath+'/'+fileName,0). La imagen del rostro se agrega a la lista facesData.

Después de leer todas las imágenes de una persona, se incrementa la etiqueta en 1 utilizando label = label + 1.

A continuación, se inicializa el reconocedor de rostros utilizando el algoritmo LBPH mediante cv2.face.LBPHFaceRecognizer_create().

Se realiza el entrenamiento del reconocedor de rostros utilizando face_recognizer.train(facesData, np.array(labels)). Los datos de los rostros y las etiquetas se pasan como argumentos.

Finalmente, se almacena el modelo entrenado en un archivo utilizando face_recognizer.write(modelName + '.xml'), donde modelName es el nombre del archivo especificado como parámetro en la función. Se muestra un mensaje indicando que el modelo ha sido almacenado.

In [ ]:
def entrenarMRF(modelName):    
    dataPath = 'C:/Users/iamra/FacialRecognition/Rostros'# Ruta donde tomara los rostros
    peopleList = os.listdir(dataPath) # Tomamos los nombres de las personas
    print('Lista de personas: ', peopleList)
    labels = [] # Donde se van a guardar las etiquetas de los rostros
    facesData = [] # Donde se van a almacecenaar todos los rostros
    label = 0
    for nameDir in peopleList: # Vamos a recorrer las carpetas de las caras de las personas
        personPath = dataPath + '/' + nameDir # Escribimos la ruta de la carpeta de la persona
        print('Leyendo las imágenes')
        for fileName in os.listdir(personPath): # Vamos a leer todas las imagenes de las personas
            print('Rostros: ', nameDir + '/' + fileName + '  Etiqueta:',label)
            labels.append(label) 
            facesData.append(cv2.imread(personPath+'/'+fileName,0)) # El cero se coloca para decir que se lea en blanco y negro
        label = label + 1 # Se suma en 1 a etiqueta cuando ya termino con una carpeta

    #face_recognizer = cv2.face.EigenFaceRecognizer_create()
    #face_recognizer = cv2.face.FisherFaceRecognizer_create()
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()

    print("Entrenando...")
    face_recognizer.train(facesData, np.array(labels)) # Entrena el algoritmo
    # Almacenando el modelo obtenido
    #face_recognizer.write('modeloEigenFace.xml')
    #face_recognizer.write('modeloFisherFace_UyM.xml') # Escribe el algoritmo en un archivo 
    face_recognizer.write(modelName + '.xml')
    print("Modelo almacenado...")
    

## 7. Método para entrenar el Modelo de Reconocimiento Emocional

La función entrenarMRE(epocas, modelName) tiene como objetivo entrenar un modelo de red neuronal convolucional (CNN) para el reconocimiento de emociones. A continuación, se proporciona una descripción de lo que hace la función:

La función utiliza el generador de datos ImageDataGenerator de Keras para preprocesar las imágenes de entrenamiento y validación. Las imágenes se normalizan dividiendo los valores de píxeles por 255.

Se configuran dos generadores de datos: train_generator para las imágenes de entrenamiento y validation_generator para las imágenes de validación. Se especifica la ruta del directorio que contiene las imágenes de entrenamiento y validación, el tamaño objetivo de las imágenes, el tamaño del lote, el modo de color (escala de grises) y el modo de clasificación (categórico).

A continuación, se crea la estructura del modelo de red neuronal utilizando el objeto Sequential. Se añaden capas convolucionales (Conv2D), capas de agrupación (MaxPooling2D), capas de regularización (Dropout) y capas completamente conectadas (Dense). La última capa de salida tiene una activación softmax para clasificar las emociones en 5 categorías diferentes.

Se compila el modelo utilizando la función de pérdida categorical_crossentropy, el optimizador Adam con una tasa de aprendizaje de 0.0001 y un decaimiento de 1e-6, y se especifica que se desea medir la precisión durante el entrenamiento.

Se calcula el número de pasos por época (steps_per_epoch) dividiendo el número total de imágenes de entrenamiento por el tamaño del lote.

A continuación, se entrena el modelo utilizando el método fit y el generador de datos de entrenamiento y validación. Se especifica el número de épocas, el generador de datos de validación y los pasos de validación.

Después de completar el entrenamiento, se guarda la estructura del modelo en formato JSON utilizando to_json() y se guarda en un archivo .json. También se guardan los pesos entrenados del modelo en un archivo .h5 utilizando save_weights().

In [2]:
def entrenarMRE(epocas, modelName):
    #Inicializamos dos generadores de datos de imágenes y realizamos una normalización
    train_data_gen = ImageDataGenerator(rescale=1./255)
    validation_data_gen = ImageDataGenerator(rescale=1./255)

    """
    Preprocesamos las imagenes de entrenamiento
    directory - ruta del directorio que tiene las imagenes de entrenamiento
    target_size - Se especifica el tamaño al que deben redimensionarse todas las imagenes
    batch_size - Se especifica el tamaño del lote de imagenes que se cargará a la vez durante el entrenamiento
    color_mode - Se especifica el modo de color de las imágenes cargadas en este caso se convertiran a escala de grises
    class_mode - Especifica el modo de clasificación de las imagenes cargadas, en este caso "categorical" para 
    indicar que las imégenes están etiquetadas por categorías y se espera que el modelo realice una clasificación de varias clases.
    """
    train_generator = train_data_gen.flow_from_directory(
        'C:/Users/iamra/FER+/fer2013plus/fer2013/train',
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode="categorical"
    )

    """
    Preprocess las imagenes de prueba
    """ 
    validation_generator = validation_data_gen.flow_from_directory(
        'C:/Users/iamra/FER+/fer2013plus/fer2013/test',
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode="categorical"
    )

    #Creamos la estructura del modelo de red neuronal y se configuran las capas

    emotion_model = Sequential() #Se crea un objeto de modelo secuencial 

    emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1))) #En esta capa se extraen características de las imágenes de entrada
    emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) #Extrae caracteristicas más complejas de las características extraídas por la capa anterior
    emotion_model.add(MaxPooling2D(pool_size=(2,2))) #Se reduce la dimensionalidad de las características y ayuda a capturar patrones invarientes a escala
    emotion_model.add(Dropout(0.25)) #Se agrega una capa para regularizar el modelo, evitando el sobreajuste, esta capa desactiva aleatoriamente el 25% de las neuronas
    #Para evitar la dependencia de ciertas caracteristicas

    emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    emotion_model.add(MaxPooling2D(pool_size=(2,2)))
    emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    emotion_model.add(MaxPooling2D(pool_size=(2,2)))
    emotion_model.add(Dropout(0.25))

    emotion_model.add(Flatten()) #Se agrega una capa de aplanamiento para convertir el tensor de caracteristicas en un vector unidimensional, preparandolo para la capa completamente conectada
    emotion_model.add(Dense(1024, activation='relu'))
    emotion_model.add(Dropout(0.5))
    emotion_model.add(Dense(5, activation='softmax')) #Se agrega la capa de salida con 4 neuronas y función de activación softmax, esta capa realiza la clasificación en 4 emociones diferentes

    emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])
    #Se compila el modelo con la función de pérdida de entropía cruzada categórica, el optimizador Adam con una taza de aprendizaaje de 0.00001 y un 
    #decaimiento de 1e-6 y se especifica que se desea medir la precisión durante el entrenamiento

    steps_per_epoch = train_generator.samples // train_generator.batch_size

    #Se entrena el modelo de la red neuronal

    """
    Se utiliza fit_generator para entrenar el modelo utilizando generadores de datos
    steps_per_epoch= total de imagenes de entrenamiento / batch_size,
    epochs - cantidad de épocas que el modelo debe entrenar
    validation_data - se pasa el generador de datos de validación 
    validation_steps= total de imagenes de validación / batch_size
    """
    emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epocas,
        validation_data=validation_generator,
        validation_steps=6824 // 64
    )


    #Guardamos la estructura del modelo en formato JSON
    model_json = emotion_model.to_json()
    with open(modelName + ".json", "w") as json_file:
        json_file.write(model_json)

    #Guardamos los pesos entrenados del modelo 

    emotion_model.save_weights(modelName + '.h5')

## 8. Método para ejecutar el sistema

La función llamarModelos(modelRE, modelRF) tiene como objetivo cargar y utilizar los modelos previamente entrenados para el reconocimiento de emociones y reconocimiento facial respectivamente. A continuación, se proporciona una descripción de lo que hace la función:

La función comienza cargando la configuración del modelo de reconocimiento de emociones desde un archivo JSON. Luego, crea el modelo utilizando la configuración cargada y carga los pesos del modelo entrenado desde un archivo .h5. El modelo cargado se compila con una función de pérdida categorical_crossentropy, un optimizador adam y se especifica que se desea medir la precisión.

A continuación, se carga la información de los datos de las personas desde los archivos de texto en la ruta especificada en datosPath. Se recopilan los nombres, cargos y edades de las personas en listas separadas.

Se carga el modelo de reconocimiento facial utilizando la función read() de OpenCV. Se especifica la ruta del archivo XML que contiene la información del modelo.

Se inicializa la captura de video utilizando la cámara. Se utiliza el clasificador frontal de Haar (haarcascade_frontalface_default.xml) para detectar rostros en el video.

Dentro del bucle principal, se procesa cada frame del video. Se redimensiona el frame y se convierte a RGB. Se utilizan los detectores de rostros y de malla facial de MediaPipe para obtener las coordenadas de los rostros y los puntos de interés en el rostro.

Si se detecta un rostro en el frame, se extrae el rostro y se realiza el reconocimiento de emociones utilizando el modelo de emociones cargado. También se realiza el reconocimiento facial utilizando el modelo de reconocimiento facial cargado. Los resultados se muestran en el frame con etiquetas de nombre, cargo, edad y estado emocional. Se dibuja un rectángulo alrededor del rostro y se dibujan los puntos de la malla facial si están disponibles.

Finalmente, se muestra el frame procesado en una ventana. El bucle continúa hasta que se presione la tecla 'Esc'.

In [13]:
def llamarModelos(modelRE, modelRF):
    # Cargar la configuración del modelo desde el archivo JSON
    with open('C:/Users/iamra/EmotionDetection/'+ modelRE +'.json', 'r') as json_file:
        loaded_model_json = json_file.read()

    # Crear el modelo a partir de la configuración cargada
    loaded_model = model_from_json(loaded_model_json)

    # Cargar los pesos del modelo entrenado
    loaded_model.load_weights('C:/Users/iamra/EmotionDetection/'+ modelRE +'.h5')

    # Compilar el modelo cargado
    loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    Datos = []
    Nombres = []
    Cargos = []
    Edades = []
    entrar_salir = [False,False]

    datosPath = 'C:/Users/iamra/FacialRecognition/Datos'
    peopleList = os.listdir(datosPath)
    for person in peopleList: 
        filePathPerson = datosPath + '/' + person
        with open(filePathPerson, "r") as file:
            Datos.append(file.readlines())


    for Dato in Datos:
        aux = Dato[0].split()
        Nombres.append(aux[1])
        aux = Dato[2].split()
        Cargos.append(aux[1])
        aux = Dato[4].split()
        Edades.append(aux[1])

    #face_recognizer = cv2.face.EigenFaceRecognizer_create()
    #face_recognizer = cv2.face.FisherFaceRecognizer_create()
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Leyendo el modelo
    #face_recognizer.read('modeloEigenFace.xml')
    #face_recognizer.read('modeloFisherFace_UyM.xml')
    face_recognizer.read('C:/Users/iamra/FacialRecognition/'+ modelRF +'.xml')

    model = loaded_model
    #emotion_dict = {0: 'Enojado', 1:'Disgustado', 2: 'Miedo', 3:'Feliz', 4:'Neutro', 5:'Triste', 6: 'Sorprendido'}
    emotion_dict = {0: 'Enojado',1:'Feliz', 2:'Neutro', 3:'Trizte', 4:'Sorprendido'}

    # inicializar el clasificador frontal de Haar para la detección de rostros
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    mp_face_detection = mp.solutions.face_detection # Es el detector de rostros
    mp_drawing = mp.solutions.drawing_utils # Esto es para colocar el recuadro y los puntos
    mp_face_mesh = mp.solutions.face_mesh

    # Crear una ventana con el nombre "Ventana"
    cv2.namedWindow("Reconocimiento F&E", cv2.WINDOW_NORMAL)

    # Cambiar el tamaño de la ventana para ajustarse a la pantalla completa
    cv2.setWindowProperty("Reconocimiento F&E", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=2,
        min_detection_confidence=0.5) as face_mesh:
        with mp_face_detection.FaceDetection(  # Gestor de contexto
            min_detection_confidence=0.6) as face_detection:

            while True:
                ret, frame = cap.read() # Comenzamos a leer el video
                if ret == False: break # Verificamos si se esta leyendo de forma correcta
                frame = imutils.resize(frame, width=640)
                frame = cv2.flip(frame, 1)
                aux_frame = frame.copy()
                height, width, _ = frame.shape # Obtenemos las dimensipones de la imagen
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2 lo lee como BGR y lo convertimos a RGB       
                results = face_detection.process(frame_rgb) # Obtenemos las cordenadas de la cara y de los puntos de interes
                malla = face_mesh.process(frame_rgb)
                k = cv2.waitKey(1) & 0xFF


                if results.detections is not None: # Si detecta un rostro entra aqui 
                    if malla.multi_face_landmarks is not None:
                        for face_landmarks in malla.multi_face_landmarks:
                            x_values = [lm.x for lm in face_landmarks.landmark]
                            y_values = [lm.y for lm in face_landmarks.landmark]
                            x_min, x_max = min(x_values), max(x_values)
                            y_min, y_max = min(y_values), max(y_values)

                            x = int(x_min * width)
                            y = int(y_min * height)
                            w = int((x_max - x_min) * width)
                            h = int((y_max - y_min) * height)

                            max_face_size = 0.8
                            if w * h > max_face_size * width * height:
                                continue


                            # Recortamos el rostro y reajustamos el tamaño   
                            rostro = aux_frame[y:y+h,x:x+w]
                            if rostro.shape[0] != 0 and rostro.shape[1] != 0:
                                face_gray = cv2.cvtColor(rostro, cv2.COLOR_BGR2GRAY)

                            try:
                                face_gray = cv2.resize(face_gray, (48,48))
                                face_gray = face_gray / 255.0
                                face_gray = np.expand_dims(face_gray, axis=-1)
                                predictions = model.predict(np.array([face_gray]))
                                emotion = emotion_dict[np.argmax(predictions)]
                                rostro = cv2.resize(rostro,(150,150), interpolation=cv2.INTER_CUBIC)
                                rostro = cv2.cvtColor(rostro, cv2.COLOR_BGR2GRAY)
                                label = face_recognizer.predict(rostro)
                                cv2.putText(frame,'{}'.format(label),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
                                coord_x_text = x-w-45
                                print(label)

                                if label[1] < 95:
                                    cv2.putText(frame,'Nombre:{}'.format(Nombres[label[0]]),(coord_x_text,y-65), cv2.LINE_AA, 0.9, (0,255,0), 2)
                                    cv2.putText(frame,'Cargo:{}'.format(Cargos[label[0]]),(coord_x_text,y-30), cv2.LINE_AA, 0.9,(0,255,0), 2)
                                    cv2.putText(frame,'Edad:{}'.format(Edades[label[0]]),(coord_x_text,y+5), cv2.LINE_AA, 0.9,(0, 255, 0), 2)
                                    cv2.putText(frame, 'Estado:{}'.format(emotion), (coord_x_text,y+35), cv2.LINE_AA, 0.9, (0, 255, 0), 2)
                                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)   
                                    mp_drawing.draw_landmarks(frame, face_landmarks,
                                        mp_face_mesh.FACEMESH_CONTOURS , #FACEMESH_TESSELATION FACEMESH_CONTOURS  FACE_CONNECTIONS
                                        mp_drawing.DrawingSpec(color=(0,255, 0), thickness=1, circle_radius=1),
                                        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1))

                                else:
                                    cv2.putText(frame,'Desconocido',(coord_x_text,y-20),cv2.LINE_AA, 0.9, (0,0,255), 2)
                                    cv2.putText(frame, 'Estado:{}'.format(emotion), (coord_x_text, y +10), cv2.LINE_AA, 0.9, (0, 0, 255), 2)
                                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
                                    mp_drawing.draw_landmarks(frame, face_landmarks,
                                        mp_face_mesh.FACEMESH_CONTOURS , #FACEMESH_TESSELATION FACEMESH_CONTOURS  FACE_CONNECTIONS
                                        mp_drawing.DrawingSpec(color=(0,0, 255), thickness=1, circle_radius=1),
                                        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1))

                            except:
                                continue
                cv2.imshow("Reconocimiento F&E", frame) # Muestra lo que esta detectando 
                if k == 27: break
    cap.release()
    cv2.destroyAllWindows()

## 9. Método para actualizar los registros del usuario

La función actualizarDatos(personName, newCargo, newEdad) tiene como objetivo actualizar los datos de una persona previamente registrada. A continuación, se proporciona una descripción de lo que hace la función:

La función comienza construyendo la ruta al archivo de texto correspondiente a la persona especificada utilizando el nombre de la persona proporcionado en el parámetro personName. Luego, se verifica si el archivo existe.

Si el archivo existe, se lee su contenido y se guarda en la variable datos. A continuación, se recorre cada línea del archivo y se comprueba si corresponde a la línea que contiene el nombre, el cargo o la edad. En caso afirmativo, se actualiza el valor correspondiente con los nuevos datos proporcionados en los parámetros newCargo y newEdad. Las demás líneas se mantienen sin cambios.

Después de actualizar los datos en el archivo, se sobrescribe el archivo con los nuevos datos actualizados. Luego, se actualizan las listas Nombres, Cargos y Edades con los datos actualizados de todos los archivos de texto en el directorio.

Finalmente, se imprime la lista actualizada de nombres, cargos y edades. Si el archivo no existe, se muestra un mensaje de que el archivo correspondiente a la persona no existe.

In [31]:
def actualizarDatos(personName, newCargo, newEdad):
    # Ruta al directorio donde se encuentran los archivos de texto
    dataPath_txt = 'C:/Users/iamra/FacialRecognition/Datos'  # Cambiar en la otra computadora

    # Ruta al archivo de la persona
    filePath = os.path.join(dataPath_txt, f'{personName}.txt')

    # Comprobar si el archivo existe
    if os.path.exists(filePath):
        # Actualizar los datos en el archivo
        with open(filePath, "r") as file:
            datos = file.readlines()

        datos_actualizados = []
        for linea in datos:
            if linea.startswith("Nombre:"):
                datos_actualizados.append(f"Nombre: {personName}\n")
            elif linea.startswith("Cargo"):
                datos_actualizados.append(f"Cargo: {newCargo}\n")
            elif linea.startswith("Edad"):
                datos_actualizados.append(f"Edad: {newEdad}\n")
            else:
                datos_actualizados.append(linea)

        with open(filePath, "w") as file:
            file.writelines(datos_actualizados)
            print(f"Datos actualizados para {personName}.")

        # Actualizar las listas Nombres, Cargos y Edades
        Nombres = []
        Cargos = []
        Edades = []

        peopleList = os.listdir(dataPath_txt)
        for person in peopleList:
            filePathPerson = os.path.join(dataPath_txt, person)
            with open(filePathPerson, "r") as file:
                datos = file.readlines()

            for linea in datos:
                if linea.startswith("Nombre:"):
                    nombre = linea.strip().split(":")[1].strip()
                elif linea.startswith("Cargo:"):
                    cargo = linea.strip().split(":")[1].strip()
                elif linea.startswith("Edad:"):
                    edad = linea.strip().split(":")[1].strip()

            Nombres.append(nombre)
            Cargos.append(cargo)
            Edades.append(edad)

        # Imprimir los datos actualizados
        print(Nombres)
        print(Cargos)
        print(Edades)
    else:
        print(f"El archivo para {personName} no existe.")


## 10. Método para eliminar registros de usuarios existentes 

La función eliminarRegistro(personName) tiene como objetivo eliminar el registro de una persona en el sistema de reconocimiento facial. A continuación, se proporciona una descripción de lo que hace la función:

La función comienza construyendo las rutas a la carpeta que contiene los rostros, el video y el archivo de texto correspondientes a la persona especificada utilizando el nombre de la persona proporcionado en el parámetro personName.

A continuación, se verifica si la carpeta existe en la ruta especificada. Si la carpeta existe, se elimina utilizando os.rmdir(carpeta_path). Se muestra un mensaje indicando que la carpeta ha sido eliminada. Si la carpeta no existe, se muestra un mensaje indicando que la carpeta no fue encontrada.

Luego, se verifica si el video existe en la ruta especificada. Si el video existe, se elimina utilizando os.remove(video_path). Se muestra un mensaje indicando que el video ha sido eliminado. Si el video no existe, se muestra un mensaje indicando que el video no fue encontrado.

Finalmente, se verifica si el archivo de texto existe en la ruta especificada. Si el archivo de texto existe, se elimina utilizando os.remove(txt_path). Se muestra un mensaje indicando que el archivo de texto ha sido eliminado. Si el archivo de texto no existe, se muestra un mensaje indicando que el archivo de texto no fue encontrado.

In [33]:
def eliminarRegistro(personName):
    # Rutas de la carpeta, video y archivo de texto
    carpeta_path = f'C:/Users/iamra/FacialRecognition/Rostros/{personName}'
    video_path = f'C:/Users/iamra/FacialRecognition/Videos/{personName}.mp4'
    txt_path = f'C:/Users/iamra/FacialRecognition/Datos/{personName}.txt'

    # Eliminar la carpeta
    if os.path.exists(carpeta_path):
        os.rmdir(carpeta_path)
        print(f"Carpeta {personName} eliminada.")
    else:
        print(f"Carpeta {personName} no encontrada.")

    # Eliminar el video
    if os.path.exists(video_path):
        os.remove(video_path)
        print(f"Video {personName}.mp4 eliminado.")
    else:
        print(f"Video {personName}.mp4 no encontrado.")

    # Eliminar el archivo de texto
    if os.path.exists(txt_path):
        os.remove(txt_path)
        print(f"Archivo {personName}.txt eliminado.")
    else:
        print(f"Archivo {personName}.txt no encontrado.")
